# Compute f2 score on train/validation dataset of a trained model

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [5]:
import numpy as np
import pandas as pd

In [6]:
from data_utils import unique_tags, get_label, GENERATED_DATA

target_tags = ['target_' + t for t in unique_tags]
val_predictions_filepath = os.path.join(GENERATED_DATA, "val_predictions.csv")
if not os.path.exists(val_predictions_filepath):
    
    val_predictions_csv = glob(os.path.join(OUTPUT_PATH, "val_predictions_*2017-07-13-22-23.csv"))
    df = pd.read_csv(val_predictions_csv[0]).dropna()
    for filepath in val_predictions_csv[1:]:
        df = pd.concat([df, pd.read_csv(filepath).dropna()])
    df.reset_index(inplace=True)   
    df.drop('index', axis=1, inplace=True)
    df['image_id'] = df['image_name'].apply(lambda x: int(x[len('train_'):]))    
    for t in target_tags:
        df[t] = ''
    def fill_target_tags(row):
        image_id = row[0]
        labels = get_label(image_id, "Train_jpg")
        row[1:] = labels    
        return row
    cols = ['image_id', ] + target_tags
    df[cols] = df[cols].apply(fill_target_tags, axis=1)
    
    df.to_csv(val_predictions_filepath, index=False)
    val_predictions_df = df
    df = None    
else:
    val_predictions_df = pd.read_csv(val_predictions_filepath)

In [10]:
len(val_predictions_df)

40320

In [9]:
best_thresholds = {}

def search_best_threshold(y_true, y_preds):
    _thr = 0.5    
    _bs = score(y_true, y_preds > _thr)
    
    thrs = np.arange(0.0, 1.0, 0.01)
    for thr in thrs:
        s = score(y_true, y_preds > thr)
        if s > _bs:
            _bs = s
            _thr = thr      
    return _thr, _bs


for tag_index, tag in enumerate(unique_tags):
    best_thresholds[tag], best_score = search_best_threshold(y_true[:, tag_index], val_predictions_df[unique_tags].values[:, tag_index])
    print("%s | best threshold : %f with score: %f" % (tag, best_thresholds[tag], best_score))


agriculture | best threshold : 0.220000 with score: 0.895218
artisinal_mine | best threshold : 0.060000 with score: 0.782789
bare_ground | best threshold : 0.090000 with score: 0.442280
blooming | best threshold : 0.090000 with score: 0.335871
blow_down | best threshold : 0.030000 with score: 0.105556
clear | best threshold : 0.200000 with score: 0.978155
cloudy | best threshold : 0.170000 with score: 0.897765
conventional_mine | best threshold : 0.070000 with score: 0.385260
cultivation | best threshold : 0.160000 with score: 0.672976
habitation | best threshold : 0.150000 with score: 0.772332
haze | best threshold : 0.140000 with score: 0.766322
partly_cloudy | best threshold : 0.220000 with score: 0.943939
primary | best threshold : 0.250000 with score: 0.990730
road | best threshold : 0.220000 with score: 0.864248
selective_logging | best threshold : 0.070000 with score: 0.420290
slash_burn | best threshold : 0.040000 with score: 0.158176
water | best threshold : 0.190000 with scor